In [ ]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# VORP positional values for bench and waiver

vorp_bench_idx = {
    'RB': 24,     
    'WR': 20,
    'TE': 8,
    'QB': 8,
    'DST': 8,
    'K': 8
}

vorp_waiver_idx = {
    'RB': 42,   # 40-44
    'WR': 46,   # 44-48
    'TE': 11,
    'QB': 10,
    'DST': 8,
    'K': 8
}


In [ ]:
df = pd.read_csv('src_data/projections_2025_wk0.csv', index_col=0)
df = df[df['position'].isin(vorp_bench_idx.keys())].copy()
df.head(10)

In [ ]:
def nth_by_position(df: pd.DataFrame, idx_map: dict, sort_col: str = "points") -> pd.DataFrame:
    """
    Return the nth (per position) highest rows based on sort_col.
    idx_map: e.g. {'RB': 22, 'WR': 18, 'TE': 8, 'QB': 8, 'D/ST': 8, 'K': 8}
    Sorts descending by sort_col (higher is better).
    """
    # ensure numeric sorting if needed
    df = df.copy()
    df[sort_col] = pd.to_numeric(df[sort_col], errors="coerce")

    # sort once
    df_sorted = df.sort_values(["position", sort_col], ascending=[True, False])

    rows = []
    for pos, grp in df_sorted.groupby("position", sort=False):
        n = idx_map.get(pos)
        if n is None:
            continue  # skip positions not in the map
        n_clamped = max(1, min(int(n + 1), len(grp)))   # clamp to available players
        rows.append(grp.iloc[n_clamped - 1])

    return pd.DataFrame(rows).reset_index(drop=True)



In [ ]:
bench_baselines = nth_by_position(df, vorp_bench_idx, sort_col="points")
waiver_baselines = nth_by_position(df, vorp_waiver_idx, sort_col="points")

# If you just want a quick lookup of baseline points per position:
bench_points = bench_baselines.set_index("position")["points"].to_dict()
waiver_points = waiver_baselines.set_index("position")["points"].to_dict()


print(bench_points)
print(waiver_points)

In [ ]:
# Example VORP column using bench baseline:
df["vorp_starter_adj"] = df.apply(
    lambda r: r["points"] - bench_points.get(r["position"]), axis=1
)

df["vorp_starter_adj_floor"] = df.apply(
    lambda r: r["floor"] - bench_points.get(r["position"]), axis=1
)
df["vorp_starter_adj_ceiling"] = df.apply(
    lambda r: r["ceiling"] - bench_points.get(r["position"]), axis=1
)

df["vorp_bench_adj"] = df.apply(
    lambda r: r["points"] - waiver_points.get(r["position"]), axis=1
)

df["vorp_bench_adj_floor"] = df.apply(
    lambda r: r["floor"] - bench_points.get(r["position"]), axis=1
)
df["vorp_bench_adj_ceiling"] = df.apply(
    lambda r: r["ceiling"] - bench_points.get(r["position"]), axis=1
)

df.head(5)

In [ ]:
df_sorted_by_pt_vor = df.sort_values(by="points_vor", ascending=False)

html_table = df_sorted_by_pt_vor.to_html(max_rows=100, max_cols=30, notebook=True)
scrollable_html = f'<div style="overflow-x:auto; height:400px; display:block;">{html_table}</div>'
display(HTML(scrollable_html))

In [ ]:
df_sorted_by_starter_vorp = df.sort_values(by="vorp_starter_adj", ascending=False)

html_table = df_sorted_by_starter_vorp.to_html(max_rows=300, max_cols=30, notebook=True)
scrollable_html = f'<div style="overflow-x:auto; height:400px; display:block;">{html_table}</div>'
display(HTML(scrollable_html))

In [ ]:
df_sorted_by_bench_vorp = df.sort_values(by="vorp_bench_adj", ascending=False)

html_table = df_sorted_by_starter_vorp.to_html(max_rows=100, max_cols=30, notebook=True)
scrollable_html = f'<div style="overflow-x:auto; height:400px; display:block;">{html_table}</div>'
display(HTML(scrollable_html))

In [ ]:
df_plot = df.copy()
for col in ["points", "floor", "ceiling"]:
    df_plot[col] = pd.to_numeric(df_plot[col], errors="coerce")
    
# 1-indexed rank within each position by *points* (highest -> 1)
df_plot["pos_rank"] = (
    df_plot.groupby("position")["points"]
           .rank(method="first", ascending=False)
           .astype(int)
)

# --- baselines (1-indexed) ---
def baseline_points(df_in, pos, nth):
    """
    Return baseline points for the nth-ranked player (1-indexed) at position `pos`.
    Clamps nth into [1, len(position_group)].
    """
    sub = df_in[df_in["position"] == pos].sort_values("points", ascending=False)
    if sub.empty:
        return None
    nth = max(1, min(int(nth), len(sub)))
    return sub.iloc[nth]["points"]   # <-- 1-indexed -> iloc[nth-1]


# precompute baseline values from the *copied* frame
bench_baselines  = {pos: baseline_points(df_plot, pos, n) for pos, n in vorp_bench_idx.items()}
waiver_baselines = {pos: baseline_points(df_plot, pos, n) for pos, n in vorp_waiver_idx.items()}

# --- plotting ---
positions = df_plot["position"].dropna().unique().tolist()
cmap = plt.cm.get_cmap("tab10", len(positions))
pos_to_color = {pos: cmap(i) for i, pos in enumerate(positions)}

plt.figure(figsize=(12, 7))

# scatter for players
positions = ['RB', 'WR', 'TE', 'QB']  

for pos in positions:
    sub = df_plot[df_plot["position"] == pos].copy()
    if sub.empty:
        continue

    # yerr = difference from mean points to floor/ceiling
    lower_err = np.clip(sub["points"] - sub["floor"],   a_min=0, a_max=None)
    upper_err = np.clip(sub["ceiling"] - sub["points"], a_min=0, a_max=None)

    # scatter points
    plt.scatter(
        sub["pos_rank"], sub["points"],
        label=pos, s=30, color=pos_to_color[pos], edgecolors="none"
    )

    # error bars showing floor -> ceiling
    plt.errorbar(
        sub["pos_rank"], sub["points"],
        yerr=[lower_err, upper_err],
        fmt="none", ecolor=pos_to_color[pos], alpha=0.55, capsize=2
    )

    # horizontal baselines for this position
    y_bench = bench_baselines.get(pos)
    if y_bench is not None:
        plt.axhline(y=y_bench, color=pos_to_color[pos], linestyle="--", alpha=0.6,
                    label=f"{pos} bench baseline")
    y_waiv = waiver_baselines.get(pos)
    if y_waiv is not None:
        plt.axhline(y=y_waiv, color=pos_to_color[pos], linestyle=":", alpha=0.6,
                    label=f"{pos} waiver baseline")


plt.xlabel("Rank within position (1 = best)")
plt.ylabel("Projected points")
plt.title("Projected Points by Position Rank with VORP baselines")
plt.legend(ncol=2, fontsize=8)
plt.grid(True, linewidth=0.4, alpha=0.5)


plt.tight_layout()
plt.show()


In [ ]:
df_plot = df.copy()
for col in ["points_vor", "floor_vor", "ceiling_vor"]:
    df_plot[col] = pd.to_numeric(df_plot[col], errors="coerce")
    
# 1-indexed rank within each position by *points* (highest -> 1)
df_plot["pos_rank"] = (
    df_plot.groupby("position")["points"]
           .rank(method="first", ascending=False)
           .astype(int)
)

# --- baselines (1-indexed) ---
def baseline_points(df_in, pos, nth):
    """
    Return baseline points for the nth-ranked player (1-indexed) at position `pos`.
    Clamps nth into [1, len(position_group)].
    """
    sub = df_in[df_in["position"] == pos].sort_values("points", ascending=False)
    if sub.empty:
        return None
    nth = max(1, min(int(nth), len(sub)))
    return sub.iloc[nth]["points"]   # <-- 1-indexed -> iloc[nth-1]


# precompute baseline values from the *copied* frame
bench_baselines  = {pos: baseline_points(df_plot, pos, n) for pos, n in vorp_bench_idx.items()}
waiver_baselines = {pos: baseline_points(df_plot, pos, n) for pos, n in vorp_waiver_idx.items()}

# --- plotting ---
positions = df_plot["position"].dropna().unique().tolist()
cmap = plt.cm.get_cmap("tab10", len(positions))
pos_to_color = {pos: cmap(i) for i, pos in enumerate(positions)}

plt.figure(figsize=(12, 7))

# scatter for players
positions = ['RB', 'WR', 'TE', 'QB']  

for pos in positions:
    sub = df_plot[df_plot["position"] == pos].copy()
    if sub.empty:
        continue

    # yerr = difference from mean points to floor/ceiling
    lower_err = np.clip(sub["points"] - sub["floor"],   a_min=0, a_max=None)
    upper_err = np.clip(sub["ceiling"] - sub["points"], a_min=0, a_max=None)

    # scatter points
    plt.scatter(
        sub["pos_rank"], sub["points"],
        label=pos, s=30, color=pos_to_color[pos], edgecolors="none"
    )

    # error bars showing floor -> ceiling
    plt.errorbar(
        sub["pos_rank"], sub["points"],
        yerr=[lower_err, upper_err],
        fmt="none", ecolor=pos_to_color[pos], alpha=0.55, capsize=2
    )

    # horizontal baselines for this position
    y_bench = bench_baselines.get(pos)
    if y_bench is not None:
        plt.axhline(y=y_bench, color=pos_to_color[pos], linestyle="--", alpha=0.6,
                    label=f"{pos} bench baseline")
    y_waiv = waiver_baselines.get(pos)
    if y_waiv is not None:
        plt.axhline(y=y_waiv, color=pos_to_color[pos], linestyle=":", alpha=0.6,
                    label=f"{pos} waiver baseline")


plt.xlabel("Rank within position (1 = best)")
plt.ylabel("Projected points")
plt.title("Projected Points by Position Rank with VORP baselines")
plt.legend(ncol=2, fontsize=8)
plt.grid(True, linewidth=0.4, alpha=0.5)


plt.tight_layout()
plt.show()


In [ ]:
df_plot = df.copy()

# ensure numeric
for col in ["vorp_starter_adj", "vorp_starter_adj_floor", "vorp_starter_adj_ceiling"]:
    df_plot[col] = pd.to_numeric(df_plot[col], errors="coerce")

# keep only non-negative VORP
df_plot = df_plot[df_plot["vorp_starter_adj"] >= 0].copy()

# rank within each position by VORP (higher -> rank 1)
df_plot["pos_rank"] = (
    df_plot.groupby("position")["vorp_starter_adj"]
           .rank(method="first", ascending=False)
           .astype(int)
)

# compute error bars (clip to avoid negatives and handle NaNs gracefully)
lower_err = np.clip(
    (df_plot["vorp_starter_adj"] - df_plot["vorp_starter_adj_floor"]).fillna(0),
    a_min=0, a_max=None
)
upper_err = np.clip(
    (df_plot["vorp_starter_adj_ceiling"] - df_plot["vorp_starter_adj"]).fillna(0),
    a_min=0, a_max=None
)

# we’ll merge these onto each sub-DataFrame by index
df_plot["err_lower"] = lower_err
df_plot["err_upper"] = upper_err

# --- color map per position (optional; remove color arg if you prefer defaults) ---
positions = df_plot["position"].dropna().unique().tolist()
cmap = plt.cm.get_cmap("tab10", len(positions))
pos_to_color = {pos: cmap(i) for i, pos in enumerate(positions)}

# --- plot ---
plt.figure(figsize=(10, 6))

positions = ['RB', 'WR', 'TE', 'QB']

for pos in positions:
    sub = df_plot[df_plot["position"] == pos].sort_values("pos_rank")

    # scatter points
    plt.scatter(
        sub["pos_rank"], sub["vorp_starter_adj"],
        label=pos, s=30, color=pos_to_color[pos], edgecolors="none"
    )

    # vertical error bars from floor to ceiling around vorp_starter_adj
    plt.errorbar(
        sub["pos_rank"], sub["vorp_starter_adj"],
        yerr=[sub["err_lower"], sub["err_upper"]],
        fmt="none", ecolor=pos_to_color[pos], alpha=0.55, capsize=2
    )

plt.xlabel("Rank within position (1 = best)")
plt.ylabel("VORP (starter-adjusted)")
plt.title("VORP by Position Rank with Floor & Ceiling Ranges")
plt.legend(title="Position")
plt.grid(True, linewidth=0.4, alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
df_plot = df_plot.copy()
df_plot["vorp_bench_adj"] = pd.to_numeric(df_plot["vorp_bench_adj"], errors="coerce")

df_plot = df_plot[df_plot["vorp_bench_adj"] >= 0]

# rank within each position by points (highest points -> rank 1)
df_plot["pos_rank"] = (
    df.groupby("position")["vorp_bench_adj"]
      .rank(method="first", ascending=False)
      .astype(int)
)

# --- color map per position using a Matplotlib colormap ---
positions = df_plot["position"].dropna().unique().tolist()
cmap = plt.cm.get_cmap("tab10", len(positions))  # pick a qualitative palette
pos_to_color = {pos: cmap(i) for i, pos in enumerate(positions)}

# --- plot ---
plt.figure(figsize=(10, 6))

for pos in positions:
    sub = df_plot[df_plot["position"] == pos]
    plt.scatter(
        sub["pos_rank"], sub["vorp_bench_adj"],
        label=pos,
        s=30,
        color=pos_to_color[pos],
        edgecolors="none"
    )

plt.xlabel("Rank within position (1 = best)")
plt.ylabel("Projected vorp_bench_adj")
plt.title("Projected Points by Position Rank")
plt.legend(title="Position")
plt.grid(True, linewidth=0.4, alpha=0.5)
plt.tight_layout()
plt.show()